In [ ]:



import matplotlib.pyplot as plt
import os
os.chdir("/path/to/work_dir")

import numpy as np
from scipy.stats import wasserstein_distance
from sklearn.metrics import mean_squared_error
# plt.style.use('ggplot')
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
inline_rc = dict(mpl.rcParams)
mpl.rcParams.update(mpl.rcParamsDefault)
import numpy as np
import re

# model_list = ['deberta-v3-base','deberta-v3-large','t03b','t0pp']
model_list = ['deberta-v3-base','deberta-v3-large','bert-base-cased','bert-large-cased']
dimension = ["length",'mass','price','temperature']
file_name = "mapping"

def partition(lst, gap):
    steps = int(len(lst)/gap)
    lst_d = []
    for i in range(steps):
        lst_d.append(lst[i*gap:(i+1)*gap])
    return lst_d
def read_result(path): 
    preds = []
    labels = []
    with open(path,'r') as f:
        results = f.readlines()
        for idx, r in enumerate(results):
            r = r.strip()
            text = r.split('\t')
            label = 1 if text[2] == 'entailment' else 0
            pred = 1 if text[3] == 'entailment' else 0
            label = -1 if text[2] == 'discard' else label
            pred = -1 if text[3] == 'discard' else pred
            labels.append(label)
            preds.append(pred)
    return labels, preds

nums = np.arange(40,160,2)
print(nums.shape)
d_list = []
pos_list = []
neg_list = []
discard_pos = -1
discard_neg = -1
for m_id, model_name in enumerate(model_list):
    pred_f = []
    pos_preds_d = []
    neg_preds_d = []
    for D in dimension:
        path = f"result/ASP_model/{model_name}-{D}-ei/wider/{file_name}.csv"
        labels,preds = read_result(path)
        pred_f.append(preds)
    for preds in pred_f:
        # preds = pred_f[0]
        label_list = partition(labels,60)
        pred_list = partition(preds,60)
        pos_label = np.array(label_list[0])
        neg_label = np.array(label_list[1])
        pos_preds = np.array([0]*60)
        neg_preds = np.array([0]*60)

        for idx,lst in enumerate(label_list):
            if idx % 2==0:
                pos_preds += pred_list[idx]
            else:
                neg_preds += pred_list[idx]

        func = wasserstein_distance

        pos_preds = pos_preds / (len(label_list)/2)
        neg_preds = neg_preds / (len(label_list)/2)

        pos_preds_d.append(pos_preds)
        neg_preds_d.append(neg_preds)

    pos_list.append(np.mean(np.array(pos_preds_d),axis=0))
    neg_list.append(np.mean(np.array(neg_preds_d),axis=0))



In [ ]:

import pickle
def read_pkl(path):
    with open(path,'rb') as f:
        data = pickle.load(f)
    return data
from sklearn import preprocessing
import numpy as np

min_max_scaler = preprocessing.MinMaxScaler()
human_label = ["Ordering",'Booster','Diminisher',
            'Negation','Comparative','Superlative']
t = file_name.split("_")[-1].capitalize()
print(t)
if t in human_label:

    pos_label = read_pkl(f"/human/pkl/{t.lower()}_pos.pkl")[40:160]
    neg_label = read_pkl(f"/human/pkl/{t.lower()}_neg.pkl")[40:160]
    pos_label = pos_label[::2]
    neg_label = neg_label[::2]

    pos_label = min_max_scaler.fit_transform(pos_label)
    pos_label = pos_label.squeeze()
    neg_label = min_max_scaler.fit_transform(neg_label)
    neg_label = neg_label.squeeze()


pos_label = np.array(pos_label)
neg_label = np.array(neg_label)

In [ ]:



plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']
plt.rcParams['font.size'] = 9
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['savefig.dpi'] = 300 
plt.rcParams['figure.dpi'] = 300 
plt.rc('font',family='Times New Roman')
plt.rcParams['font.size'] = 15
plt.figure(figsize=(6.17,2.4))

bert_mark = 'None'
deberta_mark = 'None'

#colors = ["#F8333C","#FCAB10",'#2B9EB3','#44AF69']

# deberta, bert color
# colors = ['#EDAE4A','#56A78E',"#8FA6CB","#4a4a69"]
# colors = ['#9492E0','#3633A7','#EF8C52','#974311']
colors = ['#9492E0','#3633A7',"#CA9C69","#6F4F2B"]

line_style = ['-','-','-','-','dotted','dotted']
empty_idx = -1
for e_idx, p in enumerate(pos_list[0]):
    if p <0:
        empty_idx = e_idx
if 'dimension' in file_name:
    pos_label[empty_idx] = 0
pos_label = np.array(pos_label) + 0.03
pos_label = np.clip(pos_label,0,1)
plt.bar(nums,pos_label,width=2,color='#BFCC94',alpha=0.4,label='label')

for idx, pos_preds in enumerate(pos_list):
    empty_idx = -1
    for e_idx, p in enumerate(pos_preds):
        if p <0:
            empty_idx = e_idx
    zorder = 8 if idx %2 ==1 else 7
    plt.plot(nums,pos_preds,marker=None,color=colors[idx],label=model_list[idx],
    lw=2.5,ls=line_style[idx],alpha=0.9,zorder=zorder)

plt.xlim([40,158])
plt.ylim([-0.05,1.05])
plt.ylabel('    ')
plt.xlabel(r"    ")

plt.xticks(np.arange(40,160,20),[f"{num:.1f}" for num in np.arange(0.4,1.5,0.2)])
plt.xticks([40,60,80,100,120,140,158],['40','60','80','100','120','140','158'])

plt.yticks([0.0,0.5,1.0],['0.0','0.5','1.0'])

plt.tight_layout()
plt.savefig(f'/plot/img/{file_name}_wider_pos.png',bbox_inches='tight',pad_inches=0.0)
plt.show()

plt.figure(figsize=(6.17,2.4))

empty_idx = -1
for e_idx, p in enumerate(neg_list[0]):
    if p <0:
        empty_idx = e_idx

if 'dimension' in file_name:
    neg_label[empty_idx] = 0  

neg_label = np.array(neg_label) + 0.03
neg_label = np.clip(neg_label,0,1)
plt.bar(nums,neg_label,width=2,color='#BFCC94',alpha=0.4,label='label')


for idx, neg_preds in enumerate(neg_list):
    empty_idx = -1
    for e_idx, p in enumerate(neg_preds):
        if p <0:
            empty_idx = e_idx
    zorder = 8 if idx %2 ==1 else 7
    plt.plot(nums,neg_preds,marker=None,color=colors[idx],label=model_list[idx],
    lw=2.5,ls=line_style[idx],alpha=0.9,zorder=zorder)


plt.xlim([40,158])
plt.ylim([-0.05,1.05])
plt.ylabel('    ')
plt.xlabel("    ")

plt.xticks(np.arange(40,160,20),[f"{num:.1f}" for num in np.arange(0.4,1.5,0.2)])
plt.xticks([40,60,80,100,120,140,158],['40','60','80','100','120','140','158'])
plt.yticks([0.0,0.5,1.0],['0.0','0.5','1.0'])

plt.tight_layout()
plt.savefig(f'plot/img/{file_name}_wider_neg.png',bbox_inches='tight',pad_inches=0.0)
plt.show()